In [1]:
%load_ext autoreload
%autoreload 2

import sys, os
import requests
import pandas as pd
import json
import talib
import numpy as np
import time
import datetime
import yfinance as yf
import warnings
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta

warnings.filterwarnings('ignore')



# Load API keys - search in multiple locations
def find_secrets():
    """Find secrets.json in current dir, parent dir, or submodules dir."""
    possible_paths = [
        "../secrets.json",  # Parent directory (markets/)
    ]
    for path in possible_paths:
        if os.path.exists(path):
            print(f"✓ Found secrets at: {path}")
            return path
    raise FileNotFoundError(
        "secrets.json not found. Searched locations:\n" + "\n".join(f"  - {p}" for p in possible_paths)
    )

SECRET_FP = find_secrets()
with open(SECRET_FP, 'r') as file:
    secrets = json.load(file)
API_KEY = secrets['eodhd_api_key']

# Import from package (works if installed with pip install -e .)
try:
    from markets.submodules import Chain, ChainHistory, Security, Index
    from markets.submodules.fa import FundamentalAnalysis
    from markets.submodules import plot
    from markets.submodules.plot import ChartStyle
    print("✓ Using installed markets package")
except ImportError:
    print("⚠ Package not installed, using local imports")
    
    # Try to find the submodules directory
    current_dir = os.getcwd()
    parent_dir = os.path.dirname(current_dir)
    
    # Check for markets/submodules (if we're in markets/implementations)
    submodules_path = os.path.join(parent_dir, 'submodules')
    
    if os.path.exists(submodules_path):
        sys.path.insert(0, submodules_path)
        print(f"  Added to path: {submodules_path}")
    
    from eodhd import Chain, ChainHistory, Security, Index, News
    import plot
    from plot import ChartStyle

cs = ChartStyle(figsize=(20,8),nyears=1, date_fmt="%Y-%m", grid=False)

✓ Found secrets at: ../secrets.json
⚠ Package not installed, using local imports
  Added to path: c:\Users\msands\OneDrive\Documents\code\markets\submodules
✓ News class created
✓ News class created


In [2]:
News().get_ticker_news(ticker='AAPL', limit=10, from_date = '2025-12-01')

,date,title,content,link,symbols,tags,sentiment
0,2025-12-10 22:36:42+00:00,Taiwan Semiconductor Stock Hits Buy Point Afte...,Taiwan Semiconductor Manufacturing posted an u...,https://finance.yahoo.com/m/6a8dc9d7-e276-3ac9...,"[1AAPL.MI, 1NVDA.MI, AAPL.BA, AAPL.MX, AAPL.US...","[EARNINGS, SEMICONDUCTORS]","{'polarity': 0, 'neg': 0, 'neu': 1, 'pos': 0}"
1,2025-12-10 21:33:57+00:00,Why Microsoft Stock Tumbled Today,Key Points\n\nA clutch of state attorneys gene...,https://www.nasdaq.com/articles/why-microsoft-...,"[AAPL.US, GOOG.US, GOOGL.US, META.US, MSFT.US]","[AI, CHILD-SAFETY, MARKETS, REGULATION, SHARE ...","{'polarity': 0.962, 'neg': 0.063, 'neu': 0.839..."
2,2025-12-10 21:05:05+00:00,"Dow, S&amp;P 500, Nasdaq rise on Fed's final r...","US stocks (^DJI, ^IXIC, ^GSPC) close Wednesday...",https://finance.yahoo.com/video/dow-p-500-nasd...,"[AAPL.US, AMZN.US, BTC-USD.US, DJI.INDX, ETH-U...","[FED, INDUSTRIALS, INTEREST-RATES, MATERIALS, ...","{'polarity': 0.966, 'neg': 0.023, 'neu': 0.894..."
3,2025-12-10 20:52:03+00:00,Attorneys general across US warn AI companies ...,[Serious boy using system AI Chatbot on mobile...,https://seekingalpha.com/news/4530257-attorney...,"[AAPL.US, ANTHRO.US, GOOG.US, GOOGL.US, META.U...","[AI-SAFETY, REGULATION, TECH]","{'polarity': -0.984, 'neg': 0.134, 'neu': 0.77..."
4,2025-12-10 19:29:39+00:00,Apple CEO Cook prods lawmakers over App Store ...,[U.S. President Trump Meets With Japanese Busi...,https://seekingalpha.com/news/4530247-apple-ce...,[AAPL.US],"[AGE-VERIFICATION, APP-STORE, REGULATION, TECH]","{'polarity': 0.4, 'neg': 0.061, 'neu': 0.849, ..."
5,2025-12-10 19:24:06+00:00,"Jeff Williams, who retired from Apple after 27...",The Walt Disney Company is looking to expand i...,https://finance.yahoo.com/news/jeff-williams-r...,"[1AAPL.MI, AAPL.BA, AAPL.MX, AAPL.US, AAPL34.S...","[AI, BOARD-NOMINATION, INNOVATION, STREAMING, ...","{'polarity': 0.995, 'neg': 0.01, 'neu': 0.889,..."
6,2025-12-10 18:13:49+00:00,"Apple's Tim Cook reportedly on Capitol Hill, O...",Yahoo Finance Senior Producer John Hyland exam...,https://finance.yahoo.com/video/apples-tim-coo...,"[1AAPL.MI, AAPL.BA, AAPL.MX, AAPL.US, AAPL34.S...","[AI, DEBT-OFFERING, EARNINGS, EARNINGS REPORT,...","{'polarity': 0.71, 'neg': 0.034, 'neu': 0.906,..."
7,2025-12-10 18:05:58+00:00,"Euronet Worldwide (EEFT) Stock Is Up, What You...",What Happened?\n\nShares of financial technolo...,https://finance.yahoo.com/news/euronet-worldwi...,"[1AAPL.MI, AAPL.BA, AAPL.MX, AAPL.US, AAPL34.S...","[CREDIT, FINANCIALS, M-A, PAYMENTS, RATES]","{'polarity': 0.992, 'neg': 0.038, 'neu': 0.826..."
8,2025-12-10 17:56:44+00:00,Apple’s Cook Presses US Lawmakers Over Child O...,(Bloomberg) -- Apple Inc. Chief Executive Offi...,https://finance.yahoo.com/news/apple-cook-pres...,"[1AAPL.MI, AAPL.BA, AAPL.MX, AAPL.US, AAPL34.S...","[PRIVACY, REGULATION, TECH]","{'polarity': 0.875, 'neg': 0.038, 'neu': 0.882..."
9,2025-12-10 17:10:39+00:00,"Talk of an AI bubble is just 'ridiculous,' thi...",Investors are still going back and forth over ...,https://finance.yahoo.com/video/talk-ai-bubble...,"[1AAPL.MI, 1GOOG.MI, 1GOOGL.MI, 1MSFT.MI, AAPL...","[AI, BUBBLES, TECH]","{'polarity': 0.758, 'neg': 0, 'neu': 0.941, 'p..."


In [3]:
News().get_latest_news( limit=10, from_date = '2025-12-01')

,date,title,content,link,symbols,tags,sentiment
0,2025-12-10 23:03:00+00:00,Ovanti Teases US Launch of Flote: A Credit Sco...,"DENVER, Dec. 10, 2025 (GLOBE NEWSWIRE) -- Ov...",https://www.globenewswire.com/news-release/202...,[OVT.US],"[CONSUMER-FINANCE, FINTECH, PAYMENTS, PRODUCT ...","{'polarity': 0.995, 'neg': 0.052, 'neu': 0.801..."
1,2025-12-10 23:01:00+00:00,Gran Tierra Energy Inc. Announces 2026 Guidanc...,Targeting Free Cash Flow of $60 to $80 Million...,https://www.globenewswire.com/news-release/202...,"[GTE.LSE, GTE.TO, GTE.US]","[CAPITAL-EXPENDITURES, COMPANY ANNOUNCEMENT, C...","{'polarity': 1, 'neg': 0.041, 'neu': 0.853, 'p..."
2,2025-12-10 23:00:46+00:00,Here are the major earnings before the open Th...,Major earnings expected before the bell on Thu...,https://seekingalpha.com/news/4530099-here-are...,"[CIEN.US, DLMAF.US, IMPP.US, LIVE.US, LOVE.US,...","[EARNINGS, EARNINGS NEWS]","{'polarity': 0.422, 'neg': 0, 'neu': 0.947, 'p..."
3,2025-12-10 23:00:00+00:00,"Bitcoin stays flat, small caps' rate-cut spike...",Yahoo Finance Markets and Data Editor Jared Bl...,https://finance.yahoo.com/video/bitcoin-stays-...,"[BTC-USD.US, CSCO.US, DJI.INDX, DX-Y.NYB, IXIC...","[BITCOIN, FED, RATE-CUT, SMALL-CAPS, TECH]","{'polarity': 0.996, 'neg': 0.01, 'neu': 0.919,..."
4,2025-12-10 23:00:00+00:00,"FI: Kirby McInerney LLP Advises Fiserv, Inc. I...","NEW YORK, Dec. 10, 2025 (GLOBE NEWSWIRE) -- ...",https://www.globenewswire.com/news-release/202...,[FI.US],"[CLASS ACTION, CLASS-ACTION, GUIDANCE, LAW & L...","{'polarity': 0.713, 'neg': 0.086, 'neu': 0.811..."
5,2025-12-10 23:00:00+00:00,Fortrea Announces Grant of Inducement Awards U...,"DURHAM, N.C., Dec. 10, 2025 (GLOBE NEWSWIRE)...",https://www.globenewswire.com/news-release/202...,[FTRE.US],"[BIOTECH, CHANGES IN COMPANY'S OWN SHARES, IND...","{'polarity': 0.989, 'neg': 0.009, 'neu': 0.84,..."
6,2025-12-10 23:00:00+00:00,Venu Holding Corporation INVESTIGATION: Kirby ...,"NEW YORK, Dec. 10, 2025 (GLOBE NEWSWIRE) -- ...",https://www.globenewswire.com/news-release/202...,[VENU.US],"[CLASS ACTION, COMPANY ANNOUNCEMENT, EARNINGS,...","{'polarity': 0.973, 'neg': 0.036, 'neu': 0.843..."
7,2025-12-10 22:53:00+00:00,D S Simon Media Launches the AI-Powered Broadc...,"NEW YORK, Dec. 10, 2025 (GLOBE NEWSWIRE) -- ...",https://www.globenewswire.com/news-release/202...,[],"[AI, AI-DISCOVERY, ARTIFICIAL INTELLIGENCE, EA...","{'polarity': 0.999, 'neg': 0.001, 'neu': 0.888..."
8,2025-12-10 22:50:00+00:00,BitMart Reshapes the Copytrading Revenue Model...,"Mahe, Seychelles, Dec. 10, 2025 (GLOBE NEWSW...",https://www.globenewswire.com/news-release/202...,[700.US],"[AUM, BITMART, BITMART EXCHANGE, BLOCKCHAIN, B...","{'polarity': 0.998, 'neg': 0.033, 'neu': 0.805..."
9,2025-12-10 22:45:14+00:00,EdgeTI announces non-brokered proposed private...,\t* Edge Total Intelligence (CTRL:CA [https:/...,https://seekingalpha.com/news/4530345-edgeti-a...,"[CTRL:CA.US, UNFYF.US]",[RATINGS],"{'polarity': 0.402, 'neg': 0.068, 'neu': 0.857..."


In [4]:
News().get_multiple_tickers_news( tickers = ['BAC', 'JPM' , 'C'], limit=4, from_date = '2025-12-01')

,date,title,content,link,symbols,tags,sentiment,primary_ticker
0,2025-12-10 22:20:07+00:00,Options plays to navigate Fed cut rates and ye...,"As US stocks (^DJI, ^IXIC, ^GSPC) continue to ...",https://finance.yahoo.com/video/options-plays-...,"[C.US, DJI.INDX, GSPC.INDX, IXIC.INDX, VSCO.US]","[BANKS, EARNINGS, EARNINGS REPORT, OPTIONS, RE...","{'polarity': 0.999, 'neg': 0.021, 'neu': 0.859...",C
1,2025-12-10 21:05:05+00:00,"Dow, S&amp;P 500, Nasdaq rise on Fed's final r...","US stocks (^DJI, ^IXIC, ^GSPC) close Wednesday...",https://finance.yahoo.com/video/dow-p-500-nasd...,"[AAPL.US, AMZN.US, BTC-USD.US, DJI.INDX, ETH-U...","[FED, INDUSTRIALS, INTEREST-RATES, MATERIALS, ...","{'polarity': 0.966, 'neg': 0.023, 'neu': 0.894...",JPM
2,2025-12-10 18:53:00+00:00,"JPMorgan Hands $1,000 to Employees Who Earn Le...",JPMorgan Chase is giving a payment of up to $1...,https://finance.yahoo.com/m/1a9bc592-14cd-3b23...,"[JPM-PC.US, JPM-PD.US, JPM.US]",[],"{'polarity': 0.25, 'neg': 0.012, 'neu': 0.967,...",JPM
3,2025-12-10 18:19:00+00:00,PNC Financial's Arm to Offer Direct Bitcoin Ac...,In a landmark move for the banking and crypto ...,https://finance.yahoo.com/news/pnc-financials-...,"[BTC-USD.US, BTC.CC, C.US, COIN.US, JPM-PC.US,...","[BANKING, BITCOIN, CRYPTO, DIGITAL-ASSETS, PAR...","{'polarity': 0.998, 'neg': 0.006, 'neu': 0.851...",JPM
4,2025-12-10 18:04:00+00:00,JPMorgan Stock Slides on Warning of Steep 2026...,Shares of JPMorgan JPM fell 4.7% in yesterday’...,https://finance.yahoo.com/news/jpmorgan-stock-...,"[700.US, C.US, JPM-PC.US, JPM-PD.US, JPM.US, K...","[BANKING, EARNINGS, FINANCIALS, FOURTH QUARTER...","{'polarity': 0.991, 'neg': 0.027, 'neu': 0.885...",JPM
5,2025-12-10 15:30:18+00:00,BC-Most Active Stocks,NEW YORK (AP) — prices for the Nasdaq composit...,https://finance.yahoo.com/news/bc-most-active-...,"[1INTC.MI, 6B0.F, 700.US, 99U.F, AAPL.US, AMD....","[EARNINGS, TECH]","{'polarity': 0.997, 'neg': 0.006, 'neu': 0.956...",BAC
6,2025-12-10 14:23:57+00:00,Citi expects this chipmaker to deliver a ’sign...,Investing.com -- Citi analysts highlighted one...,https://finance.yahoo.com/news/citi-expects-ch...,"[1MU.MI, C.MX, C.US, CTGP34.SA, MTE.F, MU.MX, ...","[AI, DRAMS, EARNINGS, PRICE TARGET, PRICE-TARG...","{'polarity': 0.796, 'neg': 0.038, 'neu': 0.903...",C
7,2025-12-10 09:13:00+00:00,Zacks Investment Ideas feature highlights: Car...,"For Immediate Release\n\nChicago, IL – Decembe...",https://finance.yahoo.com/news/zacks-investmen...,"[BAC-PB.US, BAC-PE.US, BAC-PK.US, BAC-PL.US, B...","[FED, FOMC, INTEREST-RATE, RATES, TREASURIES]","{'polarity': 0.998, 'neg': 0.038, 'neu': 0.822...",BAC
8,2025-12-09 21:00:44+00:00,"Stock market today: Dow, S&amp;P 500, Nasdaq f...",US stocks were mixed on Tuesday as the Federal...,https://finance.yahoo.com/news/live/stock-mark...,"[AVGO.US, BAC.US, BTC-USD.US, COST.US, CTO.BE,...","[AI, CAPITAL EMPLOYED, EARNINGS, EARNINGS GROW...","{'polarity': 0.999, 'neg': 0.067, 'neu': 0.83,...",BAC
9,2025-12-09 21:00:44+00:00,Stock market today: Dow slips as JPMorgan slid...,US stocks were mixed on Tuesday as the Federal...,https://finance.yahoo.com/news/live/stock-mark...,"[33L.BE, 33L.F, ARES.US, AVGO.US, BAC.US, COST...","[AI, CAPITAL EMPLOYED, EARNINGS, EARNINGS GROW...","{'polarity': 0.999, 'neg': 0.063, 'neu': 0.835...",BAC


In [5]:
News().search_news(keywords='Fed', from_date = '2025-11-01', to_date = datetime.today().strftime('%Y-%m-%d'))

,date,title,content,link,symbols,tags,sentiment
0,2025-12-10 23:00:00+00:00,Venu Holding Corporation INVESTIGATION: Kirby ...,"NEW YORK, Dec. 10, 2025 (GLOBE NEWSWIRE) -- ...",https://www.globenewswire.com/news-release/202...,[VENU.US],"[CLASS ACTION, COMPANY ANNOUNCEMENT, EARNINGS,...","{'polarity': 0.973, 'neg': 0.036, 'neu': 0.843..."
1,2025-12-10 23:00:00+00:00,"Bitcoin stays flat, small caps' rate-cut spike...",Yahoo Finance Markets and Data Editor Jared Bl...,https://finance.yahoo.com/video/bitcoin-stays-...,"[BTC-USD.US, CSCO.US, DJI.INDX, DX-Y.NYB, IXIC...","[BITCOIN, FED, RATE-CUT, SMALL-CAPS, TECH]","{'polarity': 0.996, 'neg': 0.01, 'neu': 0.919,..."
2,2025-12-10 22:45:00+00:00,When do you exit a private equity investment? ...,As part of Yahoo Finance's exclusive coverage ...,https://finance.yahoo.com/video/exit-private-e...,"[APO-PA.US, APO.US, DJI.INDX, FOMC.US, GSPC.IN...","[FINANCIAL-SERVICES, INDUSTRIALS, INTEREST-RAT...","{'polarity': 0.999, 'neg': 0.02, 'neu': 0.851,..."
3,2025-12-10 22:33:50+00:00,"Dow Jones Futures: Oracle Skids, 'Neutrality' ...",The stock market rallied toward highs on Fed c...,https://finance.yahoo.com/m/932fb9b7-9b2b-3f29...,"[1NVDA.MI, 49V.F, DJI.INDX, FIX.US, GSPC.INDX,...","[EARNINGS, FED, TECH]","{'polarity': 0, 'neg': 0, 'neu': 1, 'pos': 0}"
4,2025-12-10 22:30:00+00:00,Natura Resources Purchases Shepherd Power; Par...,Agreement will align Natura's breakthrough rea...,https://finance.yahoo.com/news/natura-resource...,"[NO8.BE, NO8.DU, NO8.MU, NOV.US]","[ADVANCED-REACTORS, ENERGY-TRANSITION, NUCLEAR...","{'polarity': 0.998, 'neg': 0.031, 'neu': 0.825..."
5,2025-12-10 22:28:00+00:00,NOV Announces Sale of Advanced Nuclear Develop...,NOV becomes investor in Natura Resources (“Nat...,https://www.globenewswire.com/news-release/202...,[NOV.US],"[ADVANCED-REACTORS, MERGERS AND ACQUISITIONS, ...","{'polarity': 0.998, 'neg': 0.023, 'neu': 0.829..."
6,2025-12-10 22:28:00+00:00,NOV Announces Sale of Advanced Nuclear Develop...,NOV Inc.\n\nNOV becomes investor in Natura Res...,https://finance.yahoo.com/news/nov-announces-s...,"[NO8.BE, NO8.DU, NO8.MU, NOV.US, NTCO3.SA]","[ADVANCED-REACTORS, ENERGY, M-A, NUCLEAR-ENERG...","{'polarity': 0.998, 'neg': 0.022, 'neu': 0.831..."
7,2025-12-10 22:20:07+00:00,Options plays to navigate Fed cut rates and ye...,"As US stocks (^DJI, ^IXIC, ^GSPC) continue to ...",https://finance.yahoo.com/video/options-plays-...,"[C.US, DJI.INDX, GSPC.INDX, IXIC.INDX, VSCO.US]","[BANKS, EARNINGS, EARNINGS REPORT, OPTIONS, RE...","{'polarity': 0.999, 'neg': 0.021, 'neu': 0.859..."
8,2025-12-10 22:19:09+00:00,US Equity Indexes Rise After Fed Unveils Timel...,US equity indexes rose on Wednesday after the ...,https://finance.yahoo.com/news/us-equity-index...,"[DJI.INDX, GSPC.INDX, IXIC.INDX]",[],"{'polarity': 0.226, 'neg': 0.041, 'neu': 0.9, ..."
9,2025-12-10 22:15:00+00:00,The Coca-Cola Company Announces CEO Succession...,Henrique Braun has been elected to become the ...,https://finance.yahoo.com/news/coca-cola-compa...,"[700.US, KO.US]","[BEVERAGE-INDUSTRY, CORPORATE-GOVERNANCE, LEAD...","{'polarity': 0.954, 'neg': 0.077, 'neu': 0.822..."


# Market Calendar

In [31]:
mktcal.get_economic_events(from_date='2025-11-01', to_date='2025-12-10', country='US', limit=1000)

,type,comparison,period,country,date,actual,previous,estimate,change,change_percentage
0,S&P Global Manufacturing PMI,None,Oct,US,2025-11-03 14:45:00,52.5,52.0,52.2,0.5,0.962
1,Construction Spending,mom,Aug,US,2025-11-03 15:00:00,NaN,-0.1,-0.1,NaN,NaN
2,ISM Manufacturing Prices,None,Oct,US,2025-11-03 15:00:00,58.0,61.9,61.7,-3.9,-6.300
3,Construction Spending,mom,Sep,US,2025-11-03 15:00:00,NaN,NaN,0.2,NaN,NaN
4,ISM Manufacturing New Orders,None,Oct,US,2025-11-03 15:00:00,49.4,48.9,49.2,0.5,1.022
...,...,...,...,...,...,...,...,...,...,...
787,CFTC Soybeans speculative net positions,None,None,US,2025-12-10 20:30:00,NaN,39.9,NaN,NaN,NaN
788,CFTC Wheat speculative net positions,None,None,US,2025-12-10 20:30:00,NaN,-70.6,NaN,NaN,NaN
789,CFTC S&P 500 speculative net positions,None,None,US,2025-12-10 20:30:00,NaN,-144.1,NaN,NaN,NaN
790,CFTC Aluminium Speculative net positions,None,None,US,2025-12-10 20:30:00,NaN,-0.1,NaN,NaN,NaN


In [32]:
mktcal.get_earnings(from_date='2025-12-01', to_date='2025-12-31')

,code,report_date,date,before_after_market,currency,actual,estimate,difference,percent
0,XPF.LSE,2025-12-01,2025-09-30,BeforeMarket,None,NaN,NaN,0.0000,NaN
1,BGACF.US,2025-12-08,2025-09-30,AfterMarket,None,NaN,NaN,0.0000,NaN
2,LMSBF.US,2025-12-08,2025-09-30,AfterMarket,None,NaN,NaN,0.0000,NaN
3,PNN.V,2025-12-08,2025-09-30,AfterMarket,None,NaN,NaN,0.0000,NaN
4,XTXXF.US,2025-12-08,2025-09-30,AfterMarket,None,-0.0826,NaN,0.0000,NaN
...,...,...,...,...,...,...,...,...,...
3574,SKIS-B.ST,2025-12-18,2025-11-30,AfterMarket,None,NaN,-5.1000,0.0000,NaN
3575,ORC.STU,2025-12-10,2025-11-30,AfterMarket,None,NaN,1.6400,0.0000,NaN
3576,HCNWF.US,2025-12-01,2025-11-30,BeforeMarket,None,NaN,NaN,0.0000,NaN
3577,TNX.TO,2025-12-02,2025-11-30,None,None,0.0000,0.0346,-0.0346,-100.0


In [33]:
mktcal.get_trends(from_date='2025-11-01', to_date='2025-12-10', symbols=['AAPL', 'NVDA'])

,code,date,period,growth,earningsEstimateAvg,earningsEstimateLow,earningsEstimateHigh,earningsEstimateYearAgoEps,earningsEstimateNumberOfAnalysts,earningsEstimateGrowth,...,revenueEstimateNumberOfAnalysts,revenueEstimateGrowth,epsTrendCurrent,epsTrend7daysAgo,epsTrend30daysAgo,epsTrend60daysAgo,epsTrend90daysAgo,epsRevisionsUpLast7days,epsRevisionsUpLast30days,epsRevisionsDownLast30days
0,AAPL.US,2017-06-30,0q,0.1060,1.5700,1.5000,1.6400,1.4200,38.0000,0.1060,...,36.00,0.0600,1.5700,1.5700,1.5700,1.5700,1.6200,2.0000,4.0000,None
1,NVDA.US,2017-07-31,0q,0.7500,0.7000,0.6600,0.7900,0.4000,23.0000,0.7500,...,28.00,0.3730,0.7000,0.6900,0.6900,0.6900,0.6800,1.0000,4.0000,None
2,AAPL.US,2017-09-30,0q,0.2470,1.8700,1.7900,2.0100,1.5000,31.0000,0.2470,...,29.00,0.0840,1.8700,1.8700,1.8700,1.8800,1.8100,0.0000,1.0000,None
3,AAPL.US,2017-09-30,+1q,0.0840,1.8100,1.4800,2.1000,1.6700,38.0000,0.0840,...,36.00,0.0500,1.8100,1.8500,1.8900,1.9000,1.9300,0.0000,2.0000,None
4,AAPL.US,2017-09-30,0y,0.0830,9.0000,8.9100,9.1500,8.3100,34.0000,0.0830,...,32.00,0.0550,9.0000,9.0100,9.0100,9.0200,8.8700,0.0000,2.0000,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173,NVDA.US,2026-04-30,+1q,1.0208,1.6369,1.4200,1.8200,0.8100,38.0000,1.0208,...,36.00,0.5934,1.6369,1.6369,1.5115,1.4804,1.4833,1.0000,33.0000,1.0000
174,AAPL.US,2026-09-30,+1y,0.0883,8.0541,7.2300,9.0000,7.3807,43.0000,0.0912,...,44.00,0.0608,8.0541,8.0274,7.9996,7.9642,7.7926,6.0000,8.0000,2.0000
175,AAPL.US,2026-09-30,0y,0.1056,8.2592,7.7600,9.0000,7.4600,40.0000,0.1071,...,40.00,0.0872,8.2592,8.2527,8.2335,7.9958,7.9578,0.0000,2.0000,4.0000
176,NVDA.US,2027-01-31,+1y,0.5912,7.4556,4.9100,9.5600,4.6854,55.0000,0.5912,...,62.00,0.4831,7.4556,7.4556,6.6837,6.3713,6.3237,2.0000,46.0000,0.0000
